In [1]:
# To pkeras_model=None training, we import the necessary functions and submodules from keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dropout, Reshape, Dense, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adadelta, SGD, RMSprop;
import keras.losses;
from keras.constraints import maxnorm;
from keras.utils import normalize, to_categorical
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
from keras import backend as K

import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, precision_recall_curve, auc

Using TensorFlow backend.


In [4]:
    import pandas as pd
    import numpy as np
    df = pd.read_csv("eif3a_full_test.csv")
    print(df)
    df.shape

      Unnamed: 0   num  V1.x  V2.x  V3.x  V4.x  V5.x  V6.x  V7.x  V8.x  ...  \
0              1     1     0     1     0     0     1     0     0     0  ...   
1              2     2     1     0     0     0     1     0     0     0  ...   
2              3     3     0     1     0     0     1     0     0     0  ...   
3              4     4     0     1     0     0     1     0     0     0  ...   
4              5     5     0     0     1     0     0     0     1     0  ...   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
1417        1418  1418     0     0     1     0     0     0     1     0  ...   
1418        1419  1419     0     1     0     0     0     0     1     0  ...   
1419        1420  1420     0     0     0     1     0     1     0     0  ...   
1420        1421  1421     0     0     1     0     0     0     1     0  ...   
1421        1422  1422     0     0     1     0     0     0     1     0  ...   

      V156.y.9  V157.y.9  V158.y.9  V159.y.9  V160.

/anaconda3/envs/DL/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (331,662,993,1324,1655,1986,2317,2648,2979,3310) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1422, 3311)

In [ ]:
    train_All_1 = df.iloc[:, 2:167]
    # train_All_1
    test_all_1 = df.iloc[0:90, 167:332]
    # test_all_1

    y_train = df.iloc[:,166:167]
    y_test = df.iloc[0:90,331:332]
    x_train = train_All_1.iloc[:,:-1]
    x_test = test_all_1.iloc[:,:-1]
    # print(y_train)
    # print(y_test)
    # print(train_All_1)
    # print(test_all_1)
    # print(x_train)

In [135]:
round = int(len(x_train.columns)/4)
lst = []
for i in range(round):
    x = x_train.iloc[:,(4*i):(4*i+4)]
    lst.append(x)
ls=[]
for j in range(round):
    x = lst[j]
    y = np.array(x).tolist()
    ls.append(y)
test = np.array(ls)
test=np.swapaxes(test,axis1=0,axis2=1)

In [136]:
round2 = int(len(x_test.columns) / 4)
lst2 = []
for i in range(round2):
    x = x_test.iloc[:, (4 * i):(4 * i + 4)]
    lst2.append(x)
ls2 = []
for j in range(round2):
    x = lst2[j]
    y = np.array(x).tolist()
    ls2.append(y)
test2 = np.array(ls2)
x_test = np.swapaxes(test2, axis1=0, axis2=1)
print(x_test.shape)

(90, 41, 4)


In [138]:
print(y_train.shape)
print(y_test.shape)

(1422, 1)
(90, 1)


In [ ]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [14]:
print(x_train.shape[1::])

(165,)


In [15]:
def build_model(x_train):
    one_filter_keras_model = Sequential()
    one_filter_keras_model.add(
        Conv1D(filters=90, kernel_size=5, padding="valid", kernel_regularizer=regularizers.l2(0.01),
               input_shape=x_train.shape))
    one_filter_keras_model.add(Activation('relu'))
    one_filter_keras_model.add(MaxPooling1D(pool_size=4, strides=2))
    one_filter_keras_model.add(Dropout(0.25))

    one_filter_keras_model.add(
        Conv1D(filters=100, kernel_size=3, padding="valid", kernel_regularizer=regularizers.l2(0.01)))
    one_filter_keras_model.add(Activation('relu'))
    one_filter_keras_model.add(MaxPooling1D(pool_size=10, strides=1))
    one_filter_keras_model.add(Dropout(0.25))

    one_filter_keras_model.add(Flatten())
    one_filter_keras_model.add(Dense(1421))
    one_filter_keras_model.add(Activation("relu"))
    one_filter_keras_model.add(Dense(1))
    one_filter_keras_model.add(Activation("sigmoid"))
    one_filter_keras_model.summary()

    adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
    one_filter_keras_model.compile(loss='binary_crossentropy', optimizer='adam',
                                   metrics=['accuracy', precision, recall])
    return one_filter_keras_model

In [141]:
def compileModel(model, x_train, x_test, y_test, y_train):
    model = model
    x_train = x_train
    x_test = x_test
    y_test = y_test
    y_train = y_train
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1)

    filepath = "weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint, earlystop]

    epoch = 2
    batchsize = 128

    history = model.fit(x_train, y_train, batch_size=batchsize, epochs=epoch,
                        validation_data=(x_test, y_test),
                        callbacks=callbacks_list)
    return history

In [139]:
model = build_model(x_train)
compileModel(model,x_train,x_test,y_test,y_train)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 1418, 90)          73890     
_________________________________________________________________
activation_5 (Activation)    (None, 1418, 90)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 708, 90)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 708, 90)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 706, 100)          27100     
_________________________________________________________________
activation_6 (Activation)    (None, 706, 100)          0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 697, 100)         

NameError: name 'compileModel' is not defined